In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import json

# SQFT Dataset

In [2]:
sqft_raw = pd.read_csv('Raw_Data/City_MedianValuePerSqft_AllHomes.csv', encoding = 'latin-1')
sqft_raw.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,6181,New York,NY,New York-Newark-Jersey City,Queens County,1,130.0,131.0,131.0,132.0,...,506,510,514,518,519,520,522,525,528,529
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,2,111.0,111.0,111.0,110.0,...,454,455,457,458,459,462,464,466,468,469
2,17426,Chicago,IL,Chicago-Naperville-Elgin,Cook County,3,87.0,88.0,89.0,89.0,...,169,170,171,171,171,171,172,173,173,173
3,39051,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,4,50.0,50.0,50.0,51.0,...,101,102,103,103,103,104,104,104,105,106
4,6915,San Antonio,TX,San Antonio-New Braunfels,Bexar County,5,54.0,54.0,53.0,53.0,...,103,104,105,105,106,106,107,108,109,109


In [3]:
# sqft_raw.Metro.loc[sqft_raw.State == 'WA']

In [4]:
# Grab only rows for specified metropolitan areas
cities = ['Seattle', 'Washington', 'Detroit', 'Denver', 'Austin', 'San Francisco', 
          'Dallas', 'New York', 'Orlando', 'Raleigh', 'Durham']
metro_index = []
for city in cities:
    metro_index.append(sqft_raw.index[sqft_raw['Metro'].str.contains(city, na = False)].tolist())
    


metro_index = [item for sublist in metro_index for item in sublist]

sqft_metro = sqft_raw.iloc[metro_index]
sqft_metro.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
23,16037,Seattle,WA,Seattle-Tacoma-Bellevue,King County,24,113.0,113.0,112.0,113.0,...,506,506,507,508,505,502,502,500,497,497
129,27362,Tacoma,WA,Seattle-Tacoma-Bellevue,Pierce County,130,84.0,84.0,85.0,84.0,...,226,228,230,232,234,235,236,238,241,244
217,13480,Renton,WA,Seattle-Tacoma-Bellevue,King County,218,89.0,88.0,89.0,88.0,...,256,257,258,259,259,258,258,257,256,256
232,3619,Bellevue,WA,Seattle-Tacoma-Bellevue,King County,233,106.0,106.0,105.0,105.0,...,446,446,449,450,449,448,449,448,446,446
326,5399,Kent,WA,Seattle-Tacoma-Bellevue,King County,327,85.0,85.0,84.0,84.0,...,213,212,213,213,213,212,213,213,212,212


In [5]:
# Set up Raleigh-Durham-Chapel Hill area
sqft_metro['Metro'] = sqft_metro['Metro'].replace(['Durham-Chapel Hill', 'Raleigh'], 'Raleigh-Durham-Chapel Hill')

C:\Users\michael\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
#Group regions under each metro area
metro_group = sqft_metro.groupby('Metro').mean()
metro_group.drop(index = ['Washington Court House', 'Austin'], columns = ['RegionID', 'SizeRank'], inplace = True)
metro_group.reset_index(inplace = True)
metro_group

,Metro,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,Austin-Round Rock,81.810811,81.837838,81.837838,81.675676,81.567568,81.756757,81.810811,81.783784,82.027027,...,156.692308,157.820513,158.743590,159.794872,160.589744,161.179487,162.051282,163.128205,164.435897,165.717949
1,Dallas-Fort Worth-Arlington,61.217391,61.268116,61.304348,61.463768,61.565217,61.710145,61.847826,61.898551,61.905797,...,122.486301,123.376712,124.479452,125.520548,126.232877,126.746575,127.198630,128.123288,129.369863,130.417808
2,Denver-Aurora-Lakewood,91.016949,91.135593,91.406780,91.627119,91.813559,92.169492,92.440678,92.898305,93.406780,...,265.800000,267.033333,268.266667,269.150000,269.900000,270.516667,271.766667,273.700000,275.816667,277.383333
3,Detroit-Warren-Dearborn,81.773196,82.206186,82.793814,83.340206,83.701031,84.278351,84.762887,85.412371,86.000000,...,135.288660,136.278351,137.360825,138.391753,138.938144,139.391753,139.525773,139.762887,140.072165,140.628866
4,New York-Newark-Jersey City,109.122164,109.148342,109.101222,108.945899,108.767888,108.708551,108.766143,108.828970,109.003490,...,268.311958,269.611785,270.689775,271.478336,272.051993,272.535529,273.194107,274.168111,275.062392,275.656846
5,Orlando-Kissimmee-Sanford,55.982759,56.051724,56.120690,55.965517,56.000000,55.965517,56.034483,56.206897,56.465517,...,133.050847,134.118644,135.169492,136.288136,136.915254,137.677966,138.457627,139.440678,140.423729,141.220339
6,Raleigh-Durham-Chapel Hill,60.710526,60.710526,60.842105,60.921053,60.921053,61.315789,61.447368,61.842105,62.105263,...,117.256410,117.666667,118.256410,118.820513,119.153846,119.435897,120.128205,121.230769,122.153846,122.769231
7,San Francisco-Oakland-Hayward,174.650602,174.710843,174.614458,174.686747,174.759036,175.060241,175.481928,176.108434,177.096386,...,723.880952,726.130952,727.833333,729.892857,732.166667,734.630952,737.559524,739.476190,738.238095,737.071429
8,Seattle-Tacoma-Bellevue,95.105263,95.052632,94.978947,94.884211,94.936842,95.136842,95.410526,95.736842,96.189474,...,283.600000,284.768421,286.568421,287.873684,287.631579,287.063158,287.442105,288.042105,288.473684,289.568421
9,Washington-Arlington-Alexandria,93.542553,93.521277,93.329787,93.063830,92.760638,92.574468,92.505319,92.468085,92.547872,...,217.905263,218.810526,219.526316,219.873684,220.173684,220.663158,221.331579,221.842105,222.315789,222.868421


In [7]:
metro_group['_1996'] = (metro_group['1996-04'] + metro_group['1996-05'] + metro_group['1996-06'] + 
                          metro_group['1996-07'] + metro_group['1996-08'] + metro_group['1996-09'] + 
                          metro_group['1996-10'] + metro_group['1996-11'] + metro_group['1996-12'])/9
metro_group['_1997'] = (metro_group['1997-01'] + metro_group['1997-02'] + metro_group['1997-03'] + 
                          metro_group['1997-04'] + metro_group['1997-05'] + metro_group['1997-06'] + 
                          metro_group['1997-07'] + metro_group['1997-08'] + metro_group['1997-09'] + 
                          metro_group['1997-10'] + metro_group['1997-11'] + metro_group['1997-12'])/12
metro_group['_1998'] = (metro_group['1998-01'] + metro_group['1998-02'] + metro_group['1998-03'] + 
                          metro_group['1998-04'] + metro_group['1998-05'] + metro_group['1998-06'] + 
                          metro_group['1998-07'] + metro_group['1998-08'] + metro_group['1998-09'] + 
                          metro_group['1998-10'] + metro_group['1998-11'] + metro_group['1998-12'])/12
metro_group['_1999'] = (metro_group['1999-01'] + metro_group['1999-02'] + metro_group['1999-03'] + 
                          metro_group['1999-04'] + metro_group['1999-05'] + metro_group['1999-06'] + 
                          metro_group['1999-07'] + metro_group['1999-08'] + metro_group['1999-09'] + 
                          metro_group['1999-10'] + metro_group['1999-11'] + metro_group['1999-12'])/12
metro_group['_2000'] = (metro_group['2000-01'] + metro_group['2000-02'] + metro_group['2000-03'] + 
                          metro_group['2000-04'] + metro_group['2000-05'] + metro_group['2000-06'] + 
                          metro_group['2000-07'] + metro_group['2000-08'] + metro_group['2000-09'] + 
                          metro_group['2000-10'] + metro_group['2000-11'] + metro_group['2000-12'])/12
metro_group['_2001'] = (metro_group['2001-01'] + metro_group['2001-02'] + metro_group['2001-03'] + 
                          metro_group['2001-04'] + metro_group['2001-05'] + metro_group['2001-06'] + 
                          metro_group['2001-07'] + metro_group['2001-08'] + metro_group['2001-09'] + 
                          metro_group['2001-10'] + metro_group['2001-11'] + metro_group['2001-12'])/12
metro_group['_2002'] = (metro_group['2002-01'] + metro_group['2002-02'] + metro_group['2002-03'] + 
                          metro_group['2002-04'] + metro_group['2002-05'] + metro_group['2002-06'] + 
                          metro_group['2002-07'] + metro_group['2002-08'] + metro_group['2002-09'] + 
                          metro_group['2002-10'] + metro_group['2002-11'] + metro_group['2002-12'])/12
metro_group['_2003'] = (metro_group['2003-01'] + metro_group['2003-02'] + metro_group['2003-03'] + 
                          metro_group['2003-04'] + metro_group['2003-05'] + metro_group['2003-06'] + 
                          metro_group['2003-07'] + metro_group['2003-08'] + metro_group['2003-09'] + 
                          metro_group['2003-10'] + metro_group['2003-11'] + metro_group['2003-12'])/12
metro_group['_2004'] = (metro_group['2004-01'] + metro_group['2004-02'] + metro_group['2004-03'] + 
                          metro_group['2004-04'] + metro_group['2004-05'] + metro_group['2004-06'] + 
                          metro_group['2004-07'] + metro_group['2004-08'] + metro_group['2004-09'] + 
                          metro_group['2004-10'] + metro_group['2004-11'] + metro_group['2004-12'])/12
metro_group['_2005'] = (metro_group['2005-01'] + metro_group['2005-02'] + metro_group['2005-03'] + 
                          metro_group['2005-04'] + metro_group['2005-05'] + metro_group['2005-06'] + 
                          metro_group['2005-07'] + metro_group['2005-08'] + metro_group['2005-09'] + 
                          metro_group['2005-10'] + metro_group['2005-11'] + metro_group['2005-12'])/12
metro_group['_2006'] = (metro_group['2006-01'] + metro_group['2006-02'] + metro_group['2006-03'] + 
                          metro_group['2006-04'] + metro_group['2006-05'] + metro_group['2006-06'] + 
                          metro_group['2006-07'] + metro_group['2006-08'] + metro_group['2006-09'] + 
                          metro_group['2006-10'] + metro_group['2006-11'] + metro_group['2006-12'])/12
metro_group['_2007'] = (metro_group['2007-01'] + metro_group['2007-02'] + metro_group['2007-03'] + 
                          metro_group['2007-04'] + metro_group['2007-05'] + metro_group['2007-06'] + 
                          metro_group['2007-07'] + metro_group['2007-08'] + metro_group['2007-09'] + 
                          metro_group['2007-10'] + metro_group['2007-11'] + metro_group['2007-12'])/12
metro_group['_2008'] = (metro_group['2008-01'] + metro_group['2008-02'] + metro_group['2008-03'] + 
                          metro_group['2008-04'] + metro_group['2008-05'] + metro_group['2008-06'] + 
                          metro_group['2008-07'] + metro_group['2008-08'] + metro_group['2008-09'] + 
                          metro_group['2008-10'] + metro_group['2008-11'] + metro_group['2008-12'])/12
metro_group['_2009'] = (metro_group['2009-01'] + metro_group['2009-02'] + metro_group['2009-03'] + 
                          metro_group['2009-04'] + metro_group['2009-05'] + metro_group['2009-06'] + 
                          metro_group['2009-07'] + metro_group['2009-08'] + metro_group['2009-09'] + 
                          metro_group['2009-10'] + metro_group['2009-11'] + metro_group['2009-12'])/12
metro_group['_2010'] = (metro_group['2010-01'] + metro_group['2010-02'] + metro_group['2010-03'] + 
                          metro_group['2010-04'] + metro_group['2010-05'] + metro_group['2010-06'] + 
                          metro_group['2010-07'] + metro_group['2010-08'] + metro_group['2010-09'] + 
                          metro_group['2010-10'] + metro_group['2010-11'] + metro_group['2010-12'])/12
metro_group['_2011'] = (metro_group['2011-01'] + metro_group['2011-02'] + metro_group['2011-03'] + 
                          metro_group['2011-04'] + metro_group['2011-05'] + metro_group['2011-06'] + 
                          metro_group['2011-07'] + metro_group['2011-08'] + metro_group['2011-09'] + 
                          metro_group['2011-10'] + metro_group['2011-11'] + metro_group['2011-12'])/12
metro_group['_2012'] = (metro_group['2012-01'] + metro_group['2012-02'] + metro_group['2012-03'] + 
                          metro_group['2012-04'] + metro_group['2012-05'] + metro_group['2012-06'] + 
                          metro_group['2012-07'] + metro_group['2012-08'] + metro_group['2012-09'] + 
                          metro_group['2012-10'] + metro_group['2012-11'] + metro_group['2012-12'])/12
metro_group['_2013'] = (metro_group['2013-01'] + metro_group['2013-02'] + metro_group['2013-03'] + 
                          metro_group['2013-04'] + metro_group['2013-05'] + metro_group['2013-06'] + 
                          metro_group['2013-07'] + metro_group['2013-08'] + metro_group['2013-09'] + 
                          metro_group['2013-10'] + metro_group['2013-11'] + metro_group['2013-12'])/12
metro_group['_2014'] = (metro_group['2014-01'] + metro_group['2014-02'] + metro_group['2014-03'] + 
                          metro_group['2014-04'] + metro_group['2014-05'] + metro_group['2014-06'] + 
                          metro_group['2014-07'] + metro_group['2014-08'] + metro_group['2014-09'] + 
                          metro_group['2014-10'] + metro_group['2014-11'] + metro_group['2014-12'])/12
metro_group['_2015'] = (metro_group['2015-01'] + metro_group['2015-02'] + metro_group['2015-03'] + 
                          metro_group['2015-04'] + metro_group['2015-05'] + metro_group['2015-06'] + 
                          metro_group['2015-07'] + metro_group['2015-08'] + metro_group['2015-09'] + 
                          metro_group['2015-10'] + metro_group['2015-11'] + metro_group['2015-12'])/12
metro_group['_2016'] = (metro_group['2016-01'] + metro_group['2016-02'] + metro_group['2016-03'] + 
                          metro_group['2016-04'] + metro_group['2016-05'] + metro_group['2016-06'] + 
                          metro_group['2016-07'] + metro_group['2016-08'] + metro_group['2016-09'] + 
                          metro_group['2016-10'] + metro_group['2016-11'] + metro_group['2016-12'])/12
metro_group['_2017'] = (metro_group['2017-01'] + metro_group['2017-02'] + metro_group['2017-03'] + 
                          metro_group['2017-04'] + metro_group['2017-05'] + metro_group['2017-06'] + 
                          metro_group['2017-07'] + metro_group['2017-08'] + metro_group['2017-09'] + 
                          metro_group['2017-10'] + metro_group['2017-11'] + metro_group['2017-12'])/12
metro_group['_2018'] = (metro_group['2018-01'] + metro_group['2018-02'] + metro_group['2018-03'] + 
                          metro_group['2018-04'] + metro_group['2018-05'] + metro_group['2018-06'] + 
                          metro_group['2018-07'] + metro_group['2018-08'] + metro_group['2018-09'] + 
                          metro_group['2018-10'] + metro_group['2018-11'] + metro_group['2018-12'])/12
metro_group.head()

,Metro,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,1996-12,...,_2009,_2010,_2011,_2012,_2013,_2014,_2015,_2016,_2017,_2018
0,Austin-Round Rock,81.810811,81.837838,81.837838,81.675676,81.567568,81.756757,81.810811,81.783784,82.027027,...,107.728632,106.474359,105.025641,105.636752,109.651709,117.337607,126.585470,137.626068,148.164530,159.801282
1,Dallas-Fort Worth-Arlington,61.217391,61.268116,61.304348,61.463768,61.565217,61.710145,61.847826,61.898551,61.905797,...,82.263218,81.505102,79.893836,79.895548,82.122717,86.678653,93.142694,102.695205,113.731164,125.376712
2,Denver-Aurora-Lakewood,91.016949,91.135593,91.406780,91.627119,91.813559,92.169492,92.440678,92.898305,93.406780,...,159.684722,157.279167,150.775000,154.081944,167.116667,181.381944,205.341667,228.533333,250.022222,269.398611
3,Detroit-Warren-Dearborn,81.773196,82.206186,82.793814,83.340206,83.701031,84.278351,84.762887,85.412371,86.000000,...,84.853952,77.894330,74.877148,79.718213,91.331615,103.409794,110.707904,118.292096,127.887457,137.694158
4,New York-Newark-Jersey City,109.122164,109.148342,109.101222,108.945899,108.767888,108.708551,108.766143,108.828970,109.003490,...,245.707391,236.388696,223.968841,216.529130,220.150000,229.889323,235.928798,242.475737,255.918833,271.163778


# Sales Count Dataset

In [8]:
us_state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY',
    'District of Columbia': 'DC'
}

In [9]:
sales_raw = pd.read_csv('Raw_Data/Sale_Counts_Seas_Adj_City.csv', encoding = 'latin-1')
sales_raw.head()

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,12447.0,Los Angeles,California,1,1626.0,1626.0,1593.0,1694.0,1768.0,1780.0,...,2913.0,2836.0,2791.0,2747.0,2678.0,2638.0,NaN,NaN,NaN,NaN
1,17426.0,Chicago,Illinois,2,3816.0,3678.0,3137.0,2778.0,2789.0,2745.0,...,4044.0,4109.0,3922.0,3775.0,3645.0,3311.0,2973.0,2833.0,2975.0,3316.0
2,39051.0,Houston,Texas,3,NaN,NaN,NaN,NaN,NaN,NaN,...,3391.0,3324.0,3329.0,3365.0,3320.0,3368.0,3329.0,3428.0,3383.0,3365.0
3,6915.0,San Antonio,Texas,4,1491.0,1408.0,1303.0,1325.0,1288.0,1258.0,...,2141.0,2023.0,1893.0,1795.0,1762.0,1763.0,1753.0,1811.0,1874.0,1874.0
4,13271.0,Philadelphia,Pennsylvania,5,2005.0,1898.0,1748.0,1685.0,1647.0,1613.0,...,2418.0,2341.0,2351.0,2314.0,2255.0,2143.0,2168.0,2171.0,2208.0,2351.0


In [10]:
# Change state names to abbreviations
sales_raw = sales_raw.replace({'StateName' : us_state_abbrev})

In [11]:
# Group regions into their metropolitan areas
sales_raw['RegionID'] = sales_raw['RegionID'].astype(int).astype(str)
sqft_metro['RegionID'] = sqft_metro['RegionID'].astype(str)

sales = list(sqft_metro['RegionID'])

sales_index = []
for sale in sales:
  if len(sales_raw.loc[sales_raw['RegionID'] == sale].index) > 0:
    sales_index.append(sales_raw.loc[sales_raw['RegionID'] == sale].index[0])

sales_metro = sales_raw.iloc[sales_index]

C:\Users\michael\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
sales_metro.head()

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
22,16037,Seattle,WA,23,876.0,745.0,589.0,557.0,544.0,517.0,...,964.0,892.0,852.0,823.0,821.0,778.0,736.0,722.0,723.0,738.0
109,27362,Tacoma,WA,110,202.0,198.0,185.0,196.0,189.0,176.0,...,404.0,390.0,394.0,406.0,408.0,378.0,354.0,357.0,370.0,362.0
170,13480,Renton,WA,171,132.0,124.0,109.0,102.0,96.0,81.0,...,207.0,207.0,184.0,198.0,195.0,186.0,167.0,153.0,161.0,162.0
184,3619,Bellevue,WA,185,NaN,NaN,NaN,NaN,NaN,NaN,...,198.0,189.0,176.0,186.0,177.0,166.0,142.0,133.0,141.0,142.0
250,5399,Kent,WA,251,125.0,105.0,87.0,77.0,80.0,76.0,...,171.0,156.0,149.0,150.0,164.0,155.0,142.0,128.0,136.0,141.0


In [13]:
# Generate list of metro area names
metro_names = list(sqft_metro.Metro.unique())
metro_names.remove('Washington Court House')
metro_names.remove('Austin')

metro_names = ['Seattle-Tacoma-Bellevue',
 'Washington-Arlington-Alexandria',
 'Detroit-Warren-Dearborn',
 'Denver-Aurora-Lakewood',
 'Austin-Round Rock',
 'Orlando-Kissimmee-Sanford',
 'Raleigh-Durham-Chapel Hill', 'Dallas-Fort Worth-Arlington', 'San Francisco-Oakland-Hayward', 'New York-Newark-Jersey City']

In [14]:
# Dictionary of metro aread and states
metro_agg = []
for i in range(0, len(metro_names)):
    state = list(sqft_metro.loc[sqft_metro.Metro == metro_names[i]].State.unique())
    metro_agg.append({metro_names[i] : state})

metro_agg

[{'Seattle-Tacoma-Bellevue': ['WA']},
 {'Washington-Arlington-Alexandria': ['DC', 'MD', 'VA', 'WV']},
 {'Detroit-Warren-Dearborn': ['MI']},
 {'Denver-Aurora-Lakewood': ['CO']},
 {'Austin-Round Rock': ['TX']},
 {'Orlando-Kissimmee-Sanford': ['FL']},
 {'Raleigh-Durham-Chapel Hill': ['NC']},
 {'Dallas-Fort Worth-Arlington': ['TX']},
 {'San Francisco-Oakland-Hayward': ['CA']},
 {'New York-Newark-Jersey City': ['NY', 'NJ']}]

In [15]:
sales_agg = pd.DataFrame()
for i in range(0, len(metro_names)):
    for j in range(0, len(metro_agg[i][metro_names[i]])):
        WA_sales = sales_metro.loc[sales_metro.StateName == metro_agg[i][metro_names[i]][j]]
        WA_metro = sqft_metro.loc[sqft_metro.State == metro_agg[i][metro_names[i]][j]]
        WA_cities = [value for value in WA_sales.RegionName.unique() if value in WA_metro.RegionName.unique()]

        # RegionID for every city in Seattle metro area
        WA_agg = WA_sales.loc[sales_metro['RegionName'].isin(WA_cities)]
        WA_agg['Metro'] = metro_names[i]
        sales_agg = sales_agg.append(WA_agg, ignore_index = True)
sales_agg.head()

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,Metro
0,16037,Seattle,WA,23,876.0,745.0,589.0,557.0,544.0,517.0,...,892.0,852.0,823.0,821.0,778.0,736.0,722.0,723.0,738.0,Seattle-Tacoma-Bellevue
1,27362,Tacoma,WA,110,202.0,198.0,185.0,196.0,189.0,176.0,...,390.0,394.0,406.0,408.0,378.0,354.0,357.0,370.0,362.0,Seattle-Tacoma-Bellevue
2,13480,Renton,WA,171,132.0,124.0,109.0,102.0,96.0,81.0,...,207.0,184.0,198.0,195.0,186.0,167.0,153.0,161.0,162.0,Seattle-Tacoma-Bellevue
3,3619,Bellevue,WA,185,NaN,NaN,NaN,NaN,NaN,NaN,...,189.0,176.0,186.0,177.0,166.0,142.0,133.0,141.0,142.0,Seattle-Tacoma-Bellevue
4,5399,Kent,WA,251,125.0,105.0,87.0,77.0,80.0,76.0,...,156.0,149.0,150.0,164.0,155.0,142.0,128.0,136.0,141.0,Seattle-Tacoma-Bellevue


In [16]:
sales_group_mean = sales_agg.groupby('Metro').mean()
sales_group_mean.drop(columns = ['SizeRank'], inplace = True)
sales_group_mean.reset_index(inplace = True)
sales_group_mean

,Metro,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,Austin-Round Rock,61.201550,56.976744,52.348837,52.689922,52.000000,51.093023,50.542636,49.062016,46.937984,...,92.877698,86.172662,84.302158,85.927007,85.700730,83.751825,82.481481,79.679104,76.061069,63.886792
1,Dallas-Fort Worth-Arlington,61.201550,56.976744,52.348837,52.689922,52.000000,51.093023,50.542636,49.062016,46.937984,...,92.877698,86.172662,84.302158,85.927007,85.700730,83.751825,82.481481,79.679104,76.061069,63.886792
2,Denver-Aurora-Lakewood,94.295455,84.590909,77.431818,76.113636,76.704545,75.954545,77.272727,79.454545,78.477273,...,118.826087,109.695652,102.173913,99.608696,98.086957,97.456522,88.577778,85.727273,82.931818,86.868421
3,Detroit-Warren-Dearborn,30.155844,31.233766,30.454545,28.974026,28.454545,27.103896,28.662338,29.311688,30.324675,...,54.133333,54.422222,56.911111,56.188889,57.066667,55.235955,53.696629,50.696629,50.261364,51.465517
4,New York-Newark-Jersey City,14.467562,14.324385,13.796421,13.185682,12.870246,12.409396,12.447427,11.673378,11.364653,...,28.856833,28.559653,27.043384,26.620915,25.753275,25.541485,24.230263,23.980132,23.963883,24.682008
5,Orlando-Kissimmee-Sanford,33.113636,30.318182,28.750000,28.318182,30.068182,29.159091,31.386364,31.181818,32.977273,...,91.708333,86.770833,86.354167,86.382979,85.638298,81.659574,80.446809,83.152174,83.152174,87.255814
6,Raleigh-Durham-Chapel Hill,84.250000,79.750000,69.437500,66.125000,65.312500,62.437500,61.375000,58.125000,54.750000,...,110.650000,111.300000,108.600000,104.600000,100.800000,100.100000,95.350000,94.000000,95.200000,116.235294
7,San Francisco-Oakland-Hayward,46.366667,44.583333,39.800000,40.250000,42.180328,43.786885,46.163934,47.721311,45.967213,...,81.492754,77.724638,72.101449,68.794118,67.426471,65.522388,61.378788,60.727273,61.109375,58.428571
8,Seattle-Tacoma-Bellevue,43.641026,39.807692,34.115385,32.282051,30.448718,29.230769,29.512821,29.333333,27.089744,...,71.378049,67.597561,64.414634,64.743902,64.756098,62.481481,58.607595,56.948052,59.000000,59.891892
9,Washington-Arlington-Alexandria,25.401575,24.606299,23.141732,22.937008,23.086614,23.259843,25.976744,27.054264,27.139535,...,50.154839,47.690323,47.096774,47.632258,48.810458,48.542484,46.467105,45.721854,45.302013,41.070796


In [17]:
sales_group_mean['_2008'] = (sales_group_mean['2008-03'] + sales_group_mean['2008-04'] + sales_group_mean['2008-05'] +
                         sales_group_mean['2008-06'] + sales_group_mean['2008-07'] + sales_group_mean['2008-08'] +
                         sales_group_mean['2008-09'] + sales_group_mean['2008-10'] + sales_group_mean['2008-11'] +
                         sales_group_mean['2008-12'])
sales_group_mean['_2009'] = (sales_group_mean['2009-01'] + sales_group_mean['2009-02'] + sales_group_mean['2009-03'] + 
                          sales_group_mean['2009-04'] + sales_group_mean['2009-05'] + sales_group_mean['2009-06'] + 
                          sales_group_mean['2009-07'] + sales_group_mean['2009-08'] + sales_group_mean['2009-09'] + 
                          sales_group_mean['2009-10'] + sales_group_mean['2009-11'] + sales_group_mean['2009-12'])
sales_group_mean['_2010'] = (sales_group_mean['2010-01'] + sales_group_mean['2010-02'] + sales_group_mean['2010-03'] + 
                          sales_group_mean['2010-04'] + sales_group_mean['2010-05'] + sales_group_mean['2010-06'] + 
                          sales_group_mean['2010-07'] + sales_group_mean['2010-08'] + sales_group_mean['2010-09'] + 
                          sales_group_mean['2010-10'] + sales_group_mean['2010-11'] + sales_group_mean['2010-12'])
sales_group_mean['_2011'] = (sales_group_mean['2011-01'] + sales_group_mean['2011-02'] + sales_group_mean['2011-03'] + 
                          sales_group_mean['2011-04'] + sales_group_mean['2011-05'] + sales_group_mean['2011-06'] + 
                          sales_group_mean['2011-07'] + sales_group_mean['2011-08'] + sales_group_mean['2011-09'] + 
                          sales_group_mean['2011-10'] + sales_group_mean['2011-11'] + sales_group_mean['2011-12'])
sales_group_mean['_2012'] = (sales_group_mean['2012-01'] + sales_group_mean['2012-02'] + sales_group_mean['2012-03'] + 
                          sales_group_mean['2012-04'] + sales_group_mean['2012-05'] + sales_group_mean['2012-06'] + 
                          sales_group_mean['2012-07'] + sales_group_mean['2012-08'] + sales_group_mean['2012-09'] + 
                          sales_group_mean['2012-10'] + sales_group_mean['2012-11'] + sales_group_mean['2012-12'])
sales_group_mean['_2013'] = (sales_group_mean['2013-01'] + sales_group_mean['2013-02'] + sales_group_mean['2013-03'] + 
                          sales_group_mean['2013-04'] + sales_group_mean['2013-05'] + sales_group_mean['2013-06'] + 
                          sales_group_mean['2013-07'] + sales_group_mean['2013-08'] + sales_group_mean['2013-09'] + 
                          sales_group_mean['2013-10'] + sales_group_mean['2013-11'] + sales_group_mean['2013-12'])
sales_group_mean['_2014'] = (sales_group_mean['2014-01'] + sales_group_mean['2014-02'] + sales_group_mean['2014-03'] + 
                          sales_group_mean['2014-04'] + sales_group_mean['2014-05'] + sales_group_mean['2014-06'] + 
                          sales_group_mean['2014-07'] + sales_group_mean['2014-08'] + sales_group_mean['2014-09'] + 
                          sales_group_mean['2014-10'] + sales_group_mean['2014-11'] + sales_group_mean['2014-12'])
sales_group_mean['_2015'] = (sales_group_mean['2015-01'] + sales_group_mean['2015-02'] + sales_group_mean['2015-03'] + 
                          sales_group_mean['2015-04'] + sales_group_mean['2015-05'] + sales_group_mean['2015-06'] + 
                          sales_group_mean['2015-07'] + sales_group_mean['2015-08'] + sales_group_mean['2015-09'] + 
                          sales_group_mean['2015-10'] + sales_group_mean['2015-11'] + sales_group_mean['2015-12'])
sales_group_mean['_2016'] = (sales_group_mean['2016-01'] + sales_group_mean['2016-02'] + sales_group_mean['2016-03'] + 
                          sales_group_mean['2016-04'] + sales_group_mean['2016-05'] + sales_group_mean['2016-06'] + 
                          sales_group_mean['2016-07'] + sales_group_mean['2016-08'] + sales_group_mean['2016-09'] + 
                          sales_group_mean['2016-10'] + sales_group_mean['2016-11'] + sales_group_mean['2016-12'])
sales_group_mean['_2017'] = (sales_group_mean['2017-01'] + sales_group_mean['2017-02'] + sales_group_mean['2017-03'] + 
                          sales_group_mean['2017-04'] + sales_group_mean['2017-05'] + sales_group_mean['2017-06'] + 
                          sales_group_mean['2017-07'] + sales_group_mean['2017-08'] + sales_group_mean['2017-09'] + 
                          sales_group_mean['2017-10'] + sales_group_mean['2017-11'] + sales_group_mean['2017-12'])
sales_group_mean['_2018'] = (sales_group_mean['2018-01'] + sales_group_mean['2018-02'] + sales_group_mean['2018-03'] + 
                          sales_group_mean['2018-04'] + sales_group_mean['2018-05'] + sales_group_mean['2018-06'] + 
                          sales_group_mean['2018-07'] + sales_group_mean['2018-08'] + sales_group_mean['2018-09'] + 
                          sales_group_mean['2018-10'] + sales_group_mean['2018-11'] + sales_group_mean['2018-12'])
sales_group_mean.head()

,Metro,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,...,_2009,_2010,_2011,_2012,_2013,_2014,_2015,_2016,_2017,_2018
0,Austin-Round Rock,61.201550,56.976744,52.348837,52.689922,52.000000,51.093023,50.542636,49.062016,46.937984,...,557.457364,546.778863,521.969466,584.040828,788.943515,831.007407,901.373982,983.848921,988.496403,1003.495203
1,Dallas-Fort Worth-Arlington,61.201550,56.976744,52.348837,52.689922,52.000000,51.093023,50.542636,49.062016,46.937984,...,557.457364,546.778863,521.969466,584.040828,788.943515,831.007407,901.373982,983.848921,988.496403,1003.495203
2,Denver-Aurora-Lakewood,94.295455,84.590909,77.431818,76.113636,76.704545,75.954545,77.272727,79.454545,78.477273,...,836.540909,771.244444,765.768116,942.043478,1149.739130,1168.195652,1251.326087,1226.978261,1265.695652,1198.366159
3,Detroit-Warren-Dearborn,30.155844,31.233766,30.454545,28.974026,28.454545,27.103896,28.662338,29.311688,30.324675,...,399.194805,436.185525,447.650610,504.462687,610.151710,554.511628,617.716958,665.677778,667.822222,650.700539
4,New York-Newark-Jersey City,14.467562,14.324385,13.796421,13.185682,12.870246,12.409396,12.447427,11.673378,11.364653,...,134.978119,147.013917,133.725055,137.810348,171.796411,172.924945,202.648707,248.332422,290.409311,313.750269


In [18]:
sales_group_median = sales_agg.groupby('Metro').median()
sales_group_median.drop(columns = ['SizeRank'], inplace = True)
sales_group_median.reset_index(inplace = True)
sales_group_median

,Metro,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,Austin-Round Rock,23.0,21.0,17.0,17.0,16.0,16.0,16.0,17.0,16.0,...,36.0,36.0,34.0,34.0,33.0,34.0,35.0,33.0,33.0,28.0
1,Dallas-Fort Worth-Arlington,23.0,21.0,17.0,17.0,16.0,16.0,16.0,17.0,16.0,...,36.0,36.0,34.0,34.0,33.0,34.0,35.0,33.0,33.0,28.0
2,Denver-Aurora-Lakewood,23.0,25.5,21.0,21.5,20.0,18.5,19.5,19.5,19.0,...,29.5,29.5,27.0,28.0,27.5,25.5,21.0,23.5,25.0,26.0
3,Detroit-Warren-Dearborn,18.0,20.0,19.0,17.0,15.0,14.0,16.0,17.0,16.0,...,35.0,40.0,39.5,39.0,37.0,34.0,31.0,31.0,31.5,31.5
4,New York-Newark-Jersey City,9.0,9.0,9.0,9.0,8.0,8.0,8.0,8.0,7.0,...,17.0,17.0,17.0,17.0,16.0,16.0,16.0,14.0,14.0,15.0
5,Orlando-Kissimmee-Sanford,10.0,9.5,9.0,8.5,11.0,11.0,11.0,10.0,10.0,...,39.5,34.0,33.0,31.0,31.0,31.0,30.0,29.5,30.5,30.0
6,Raleigh-Durham-Chapel Hill,22.0,20.5,20.0,18.0,15.0,14.5,15.0,15.0,14.5,...,36.5,37.5,42.5,40.5,38.5,37.5,40.0,37.5,32.5,42.0
7,San Francisco-Oakland-Hayward,26.0,22.0,18.5,20.5,23.0,22.0,20.0,22.0,19.0,...,46.0,39.0,40.0,42.0,40.0,37.0,34.5,37.0,38.0,38.0
8,Seattle-Tacoma-Bellevue,20.5,17.5,17.0,16.0,14.5,14.0,14.0,13.5,11.5,...,36.5,35.0,33.5,33.5,30.5,31.0,28.0,29.0,29.0,32.0
9,Washington-Arlington-Alexandria,11.0,10.0,10.0,9.0,9.0,8.0,9.0,10.0,11.0,...,22.0,20.0,18.0,18.0,20.0,20.0,21.0,20.0,20.0,17.0


In [19]:
sales_group_median['_2008'] = (sales_group_median['2008-03'] + sales_group_median['2008-04'] + sales_group_median['2008-05'] +
                         sales_group_median['2008-06'] + sales_group_median['2008-07'] + sales_group_median['2008-08'] +
                         sales_group_median['2008-09'] + sales_group_median['2008-10'] + sales_group_median['2008-11'] +
                         sales_group_median['2008-12'])
sales_group_median['_2009'] = (sales_group_median['2009-01'] + sales_group_median['2009-02'] + sales_group_median['2009-03'] + 
                          sales_group_median['2009-04'] + sales_group_median['2009-05'] + sales_group_median['2009-06'] + 
                          sales_group_median['2009-07'] + sales_group_median['2009-08'] + sales_group_median['2009-09'] + 
                          sales_group_median['2009-10'] + sales_group_median['2009-11'] + sales_group_median['2009-12'])
sales_group_median['_2010'] = (sales_group_median['2010-01'] + sales_group_median['2010-02'] + sales_group_median['2010-03'] + 
                          sales_group_median['2010-04'] + sales_group_median['2010-05'] + sales_group_median['2010-06'] + 
                          sales_group_median['2010-07'] + sales_group_median['2010-08'] + sales_group_median['2010-09'] + 
                          sales_group_median['2010-10'] + sales_group_median['2010-11'] + sales_group_median['2010-12'])
sales_group_median['_2011'] = (sales_group_median['2011-01'] + sales_group_median['2011-02'] + sales_group_median['2011-03'] + 
                          sales_group_median['2011-04'] + sales_group_median['2011-05'] + sales_group_median['2011-06'] + 
                          sales_group_median['2011-07'] + sales_group_median['2011-08'] + sales_group_median['2011-09'] + 
                          sales_group_median['2011-10'] + sales_group_median['2011-11'] + sales_group_median['2011-12'])
sales_group_median['_2012'] = (sales_group_median['2012-01'] + sales_group_median['2012-02'] + sales_group_median['2012-03'] + 
                          sales_group_median['2012-04'] + sales_group_median['2012-05'] + sales_group_median['2012-06'] + 
                          sales_group_median['2012-07'] + sales_group_median['2012-08'] + sales_group_median['2012-09'] + 
                          sales_group_median['2012-10'] + sales_group_median['2012-11'] + sales_group_median['2012-12'])
sales_group_median['_2013'] = (sales_group_median['2013-01'] + sales_group_median['2013-02'] + sales_group_median['2013-03'] + 
                          sales_group_median['2013-04'] + sales_group_median['2013-05'] + sales_group_median['2013-06'] + 
                          sales_group_median['2013-07'] + sales_group_median['2013-08'] + sales_group_median['2013-09'] + 
                          sales_group_median['2013-10'] + sales_group_median['2013-11'] + sales_group_median['2013-12'])
sales_group_median['_2014'] = (sales_group_median['2014-01'] + sales_group_median['2014-02'] + sales_group_median['2014-03'] + 
                          sales_group_median['2014-04'] + sales_group_median['2014-05'] + sales_group_median['2014-06'] + 
                          sales_group_median['2014-07'] + sales_group_median['2014-08'] + sales_group_median['2014-09'] + 
                          sales_group_median['2014-10'] + sales_group_median['2014-11'] + sales_group_median['2014-12'])
sales_group_median['_2015'] = (sales_group_median['2015-01'] + sales_group_median['2015-02'] + sales_group_median['2015-03'] + 
                          sales_group_median['2015-04'] + sales_group_median['2015-05'] + sales_group_median['2015-06'] + 
                          sales_group_median['2015-07'] + sales_group_median['2015-08'] + sales_group_median['2015-09'] + 
                          sales_group_median['2015-10'] + sales_group_median['2015-11'] + sales_group_median['2015-12'])
sales_group_median['_2016'] = (sales_group_median['2016-01'] + sales_group_median['2016-02'] + sales_group_median['2016-03'] + 
                          sales_group_median['2016-04'] + sales_group_median['2016-05'] + sales_group_median['2016-06'] + 
                          sales_group_median['2016-07'] + sales_group_median['2016-08'] + sales_group_median['2016-09'] + 
                          sales_group_median['2016-10'] + sales_group_median['2016-11'] + sales_group_median['2016-12'])
sales_group_median['_2017'] = (sales_group_median['2017-01'] + sales_group_median['2017-02'] + sales_group_median['2017-03'] + 
                          sales_group_median['2017-04'] + sales_group_median['2017-05'] + sales_group_median['2017-06'] + 
                          sales_group_median['2017-07'] + sales_group_median['2017-08'] + sales_group_median['2017-09'] + 
                          sales_group_median['2017-10'] + sales_group_median['2017-11'] + sales_group_median['2017-12'])
sales_group_median['_2018'] = (sales_group_median['2018-01'] + sales_group_median['2018-02'] + sales_group_median['2018-03'] + 
                          sales_group_median['2018-04'] + sales_group_median['2018-05'] + sales_group_median['2018-06'] + 
                          sales_group_median['2018-07'] + sales_group_median['2018-08'] + sales_group_median['2018-09'] + 
                          sales_group_median['2018-10'] + sales_group_median['2018-11'] + sales_group_median['2018-12'])
sales_group_median.head()

,Metro,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,...,_2009,_2010,_2011,_2012,_2013,_2014,_2015,_2016,_2017,_2018
0,Austin-Round Rock,23.0,21.0,17.0,17.0,16.0,16.0,16.0,17.0,16.0,...,192.0,198.0,189.0,219.0,265.5,285.0,329.5,385.0,379.0,412.0
1,Dallas-Fort Worth-Arlington,23.0,21.0,17.0,17.0,16.0,16.0,16.0,17.0,16.0,...,192.0,198.0,189.0,219.0,265.5,285.0,329.5,385.0,379.0,412.0
2,Denver-Aurora-Lakewood,23.0,25.5,21.0,21.5,20.0,18.5,19.5,19.5,19.0,...,249.0,245.0,247.0,296.5,312.5,329.5,336.0,323.5,338.5,328.0
3,Detroit-Warren-Dearborn,18.0,20.0,19.0,17.0,15.0,14.0,16.0,17.0,16.0,...,216.0,258.0,262.5,282.5,367.0,352.0,384.0,437.5,452.5,421.0
4,New York-Newark-Jersey City,9.0,9.0,9.0,9.0,8.0,8.0,8.0,8.0,7.0,...,85.0,96.5,86.0,93.0,112.0,117.0,131.0,153.5,181.0,191.0


# Median Sale Prices Dataset

In [20]:
price_raw = pd.read_csv('Raw_Data/Sale_Prices_Zip.csv', encoding = 'latin-1')
price_raw.rename(columns = {'RegionName':'ZipCode'}, inplace = True)
price_raw.head()

,RegionID,ZipCode,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,61639.0,10025,New York,1,NaN,NaN,NaN,NaN,NaN,NaN,...,885100.0,1076100.0,1086300.0,1053100.0,952800.0,1007400.0,1108000.0,1159900.0,1083400.0,968600.0
1,84654.0,60657,Illinois,2,NaN,NaN,NaN,NaN,NaN,NaN,...,381600.0,340200.0,334100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,91982.0,77494,Texas,3,NaN,NaN,NaN,NaN,NaN,NaN,...,336100.0,333700.0,330600.0,333900.0,322300.0,321400.0,309200.0,311300.0,304300.0,NaN
3,93144.0,79936,Texas,4,NaN,NaN,NaN,NaN,NaN,NaN,...,128100.0,126500.0,126700.0,131000.0,131900.0,130800.0,128900.0,130100.0,128400.0,126900.0
4,91940.0,77449,Texas,5,NaN,NaN,NaN,NaN,NaN,NaN,...,180100.0,182100.0,178100.0,178600.0,178700.0,180800.0,184500.0,187400.0,191000.0,190700.0


### Aggregate list of zip codes by metro area

In [21]:
zip_raw = pd.read_csv('Raw_Data/Zip_Zhvi_AllHomes.csv', encoding = 'latin-1')
zip_raw.rename(columns = {'RegionName':'ZipCode'}, inplace = True)
zip_raw.head()

,RegionID,ZipCode,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,171600.0,171600.0,171400.0,...,1122300,1117600,1115600,1110000,1098000,1086700,1080400,1072600,1063400,1058600
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,158400.0,159700.0,160700.0,...,352700,351400,349900,348300,347200,347400,348800,350900,352700,353500
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,347900.0,349600.0,351100.0,...,1496600,1494700,1502400,1489400,1462300,1438000,1411200,1390300,1378900,1370500
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,210400.0,212200.0,212200.0,...,330400,332700,334500,335900,337000,338300,338400,336900,336000,336500
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,192500.0,194500.0,196100.0,...,430100,428800,427900,428200,429900,430900,430000,430200,431500,432300


In [22]:
# filter zip codes by metro area
zip_metro = zip_raw.loc[zip_raw['Metro'].isin(metro_names)]

# Create Realeigh-Durham-Chapel Hill area
zip_NC = zip_raw.loc[zip_raw['Metro'].isin(['Raleigh', 'Durham-Chapel Hill'])]
zip_NC['Metro'] = 'Raleigh-Durham-Chapel Hill'
zip_metro = zip_metro.append(zip_NC)

# Create dictionary of zip codes by metro area
zip_agg = []
for i in range(0, len(metro_names)):
    zip_agg.append({metro_names[i]:list((zip_metro.loc[zip_metro['Metro'] == metro_names[i]].ZipCode).values)})

C:\Users\michael\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
price_agg = pd.DataFrame()

# metro area, all zips, single zip
for i in range(0, len(metro_names)):
    zipper = zip_agg[i][metro_names[i]]

    metro_zips = price_raw.loc[price_raw['ZipCode'].isin(zipper)]
    metro_zips['Metro'] = metro_names[i]
    price_agg = price_agg.append(metro_zips)
price_agg.head()

C:\Users\michael\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,RegionID,ZipCode,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,Metro
38,99534.0,98052,Washington,39,429000.0,428800.0,419600.0,414600.0,410600.0,453200.0,...,759500.0,771100.0,773200.0,743000.0,751500.0,785400.0,822100.0,808100.0,782600.0,Seattle-Tacoma-Bellevue
93,99500.0,98012,Washington,94,385400.0,375800.0,366600.0,374700.0,367100.0,371300.0,...,581000.0,588700.0,589700.0,589900.0,578400.0,557200.0,536000.0,539200.0,551300.0,Seattle-Tacoma-Bellevue
117,99563.0,98103,Washington,118,NaN,NaN,NaN,NaN,NaN,NaN,...,776600.0,804200.0,795400.0,755400.0,740500.0,768300.0,805700.0,775000.0,732200.0,Seattle-Tacoma-Bellevue
249,99589.0,98133,Washington,250,318000.0,338800.0,321400.0,336500.0,331600.0,324200.0,...,544200.0,531700.0,558300.0,565000.0,550800.0,551600.0,546300.0,572400.0,560400.0,Seattle-Tacoma-Bellevue
261,99581.0,98122,Washington,262,NaN,NaN,NaN,NaN,NaN,NaN,...,768600.0,762400.0,754800.0,753300.0,788900.0,760200.0,728100.0,717300.0,684400.0,Seattle-Tacoma-Bellevue


In [24]:
price_agg = price_agg.groupby('Metro').mean()
price_agg.drop(columns = ['SizeRank', 'ZipCode', 'RegionID'], inplace = True)
price_agg.reset_index(inplace = True)
price_agg

,Metro,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12
0,Austin-Round Rock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.975300e+05,2.955833e+05,2.945167e+05,2.966867e+05,2.975900e+05,2.995333e+05,2.999867e+05,3.010467e+05,3.026200e+05,304334.482759
1,Dallas-Fort Worth-Arlington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.717054e+05,2.741860e+05,2.750000e+05,2.742839e+05,2.745538e+05,2.733043e+05,2.752413e+05,2.768297e+05,2.722071e+05,270337.681159
2,Denver-Aurora-Lakewood,233650.000000,230470.000000,227670.000000,226726.000000,227288.000000,230636.000000,224880.769231,223971.153846,222001.923077,...,4.136675e+05,4.155301e+05,4.150470e+05,4.157639e+05,4.145169e+05,4.183819e+05,4.179829e+05,4.178438e+05,4.196566e+05,416952.857143
3,Detroit-Warren-Dearborn,169489.285714,165067.857143,157925.000000,152403.571429,150875.000000,148664.285714,149807.142857,143657.142857,140957.142857,...,2.138714e+05,2.136162e+05,2.140295e+05,2.097221e+05,2.089356e+05,2.090621e+05,2.098814e+05,2.101119e+05,2.097360e+05,223560.273973
4,New York-Newark-Jersey City,395665.079365,395523.809524,390185.714286,385471.428571,384285.937500,380790.625000,377203.125000,370892.187500,368981.250000,...,4.408734e+05,4.466578e+05,4.507270e+05,4.535404e+05,4.519833e+05,4.488907e+05,4.489500e+05,4.487438e+05,4.583651e+05,475043.795620
5,Orlando-Kissimmee-Sanford,222456.521739,215817.391304,204995.652174,199504.347826,192886.956522,191265.217391,189291.304348,183317.391304,178069.565217,...,2.520400e+05,2.496217e+05,2.490933e+05,2.466867e+05,2.456217e+05,2.474283e+05,2.451356e+05,2.459000e+05,2.473684e+05,252080.357143
6,Raleigh-Durham-Chapel Hill,191500.000000,189225.000000,189325.000000,187516.666667,189650.000000,188741.666667,192383.333333,192425.000000,189991.666667,...,2.725688e+05,2.801500e+05,2.781875e+05,2.770094e+05,2.737781e+05,2.734062e+05,2.722387e+05,2.736161e+05,2.755677e+05,285255.172414
7,San Francisco-Oakland-Hayward,609295.744681,588831.914894,560665.957447,548961.702128,549729.166667,550302.083333,543089.583333,534577.083333,525085.416667,...,1.006742e+06,1.016325e+06,1.019273e+06,1.018428e+06,1.028126e+06,1.027653e+06,1.046116e+06,1.051562e+06,1.060137e+06,986753.968254
8,Seattle-Tacoma-Bellevue,369272.549020,361194.117647,353547.058824,346505.882353,344833.333333,344662.745098,339523.529412,336143.137255,329049.019608,...,5.283257e+05,5.295876e+05,5.354770e+05,5.316080e+05,5.269223e+05,5.187486e+05,5.165220e+05,5.168491e+05,5.184075e+05,521820.754717
9,Washington-Arlington-Alexandria,436698.245614,424473.684211,414382.456140,403977.192982,400580.701754,394132.758621,390791.379310,386372.881356,381777.966102,...,4.343282e+05,4.392466e+05,4.367706e+05,4.351454e+05,4.326613e+05,4.360067e+05,4.398221e+05,4.402337e+05,4.442453e+05,419747.706422


In [25]:
price_agg['_2008'] = (price_agg['2008-03'] + price_agg['2008-04'] + price_agg['2008-05'] +
                         price_agg['2008-06'] + price_agg['2008-07'] + price_agg['2008-08'] +
                         price_agg['2008-09'] + price_agg['2008-10'] + price_agg['2008-11'] +
                         price_agg['2008-12'])
price_agg['_2009'] = (price_agg['2009-01'] + price_agg['2009-02'] + price_agg['2009-03'] + 
                          price_agg['2009-04'] + price_agg['2009-05'] + price_agg['2009-06'] + 
                          price_agg['2009-07'] + price_agg['2009-08'] + price_agg['2009-09'] + 
                          price_agg['2009-10'] + price_agg['2009-11'] + price_agg['2009-12'])
price_agg['_2010'] = (price_agg['2010-01'] + price_agg['2010-02'] + price_agg['2010-03'] + 
                          price_agg['2010-04'] + price_agg['2010-05'] + price_agg['2010-06'] + 
                          price_agg['2010-07'] + price_agg['2010-08'] + price_agg['2010-09'] + 
                          price_agg['2010-10'] + price_agg['2010-11'] + price_agg['2010-12'])
price_agg['_2011'] = (price_agg['2011-01'] + price_agg['2011-02'] + price_agg['2011-03'] + 
                          price_agg['2011-04'] + price_agg['2011-05'] + price_agg['2011-06'] + 
                          price_agg['2011-07'] + price_agg['2011-08'] + price_agg['2011-09'] + 
                          price_agg['2011-10'] + price_agg['2011-11'] + price_agg['2011-12'])
price_agg['_2012'] = (price_agg['2012-01'] + price_agg['2012-02'] + price_agg['2012-03'] + 
                          price_agg['2012-04'] + price_agg['2012-05'] + price_agg['2012-06'] + 
                          price_agg['2012-07'] + price_agg['2012-08'] + price_agg['2012-09'] + 
                          price_agg['2012-10'] + price_agg['2012-11'] + price_agg['2012-12'])
price_agg['_2013'] = (price_agg['2013-01'] + price_agg['2013-02'] + price_agg['2013-03'] + 
                          price_agg['2013-04'] + price_agg['2013-05'] + price_agg['2013-06'] + 
                          price_agg['2013-07'] + price_agg['2013-08'] + price_agg['2013-09'] + 
                          price_agg['2013-10'] + price_agg['2013-11'] + price_agg['2013-12'])
price_agg['_2014'] = (price_agg['2014-01'] + price_agg['2014-02'] + price_agg['2014-03'] + 
                          price_agg['2014-04'] + price_agg['2014-05'] + price_agg['2014-06'] + 
                          price_agg['2014-07'] + price_agg['2014-08'] + price_agg['2014-09'] + 
                          price_agg['2014-10'] + price_agg['2014-11'] + price_agg['2014-12'])
price_agg['_2015'] = (price_agg['2015-01'] + price_agg['2015-02'] + price_agg['2015-03'] + 
                          price_agg['2015-04'] + price_agg['2015-05'] + price_agg['2015-06'] + 
                          price_agg['2015-07'] + price_agg['2015-08'] + price_agg['2015-09'] + 
                          price_agg['2015-10'] + price_agg['2015-11'] + price_agg['2015-12'])
price_agg['_2016'] = (price_agg['2016-01'] + price_agg['2016-02'] + price_agg['2016-03'] + 
                          price_agg['2016-04'] + price_agg['2016-05'] + price_agg['2016-06'] + 
                          price_agg['2016-07'] + price_agg['2016-08'] + price_agg['2016-09'] + 
                          price_agg['2016-10'] + price_agg['2016-11'] + price_agg['2016-12'])
price_agg['_2017'] = (price_agg['2017-01'] + price_agg['2017-02'] + price_agg['2017-03'] + 
                          price_agg['2017-04'] + price_agg['2017-05'] + price_agg['2017-06'] + 
                          price_agg['2017-07'] + price_agg['2017-08'] + price_agg['2017-09'] + 
                          price_agg['2017-10'] + price_agg['2017-11'] + price_agg['2017-12'])
price_agg['_2018'] = (price_agg['2018-01'] + price_agg['2018-02'] + price_agg['2018-03'] + 
                          price_agg['2018-04'] + price_agg['2018-05'] + price_agg['2018-06'] + 
                          price_agg['2018-07'] + price_agg['2018-08'] + price_agg['2018-09'] + 
                          price_agg['2018-10'] + price_agg['2018-11'] + price_agg['2018-12'])
# price_agg.dropna(inplace = True)
price_agg

,Metro,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,2008-11,...,_2009,_2010,_2011,_2012,_2013,_2014,_2015,_2016,_2017,_2018
0,Austin-Round Rock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.181072e+06,3.062327e+06,3.052911e+06,3.269537e+06,3.454693e+06,3.582968e+06
1,Dallas-Fort Worth-Arlington,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.960295e+06,2.977698e+06,2.830928e+06,2.894536e+06,3.093001e+06,3.272260e+06
2,Denver-Aurora-Lakewood,233650.000000,230470.000000,227670.000000,226726.000000,227288.000000,230636.000000,224880.769231,223971.153846,222001.923077,...,2.636127e+06,2.706615e+06,2.682097e+06,2.833528e+06,3.111793e+06,3.366779e+06,3.777083e+06,4.195968e+06,4.574975e+06,4.980854e+06
3,Detroit-Warren-Dearborn,169489.285714,165067.857143,157925.000000,152403.571429,150875.000000,148664.285714,149807.142857,143657.142857,140957.142857,...,1.497827e+06,1.379841e+06,1.332955e+06,1.531673e+06,1.828872e+06,2.025717e+06,2.143266e+06,2.220444e+06,2.397070e+06,2.543773e+06
4,New York-Newark-Jersey City,395665.079365,395523.809524,390185.714286,385471.428571,384285.937500,380790.625000,377203.125000,370892.187500,368981.250000,...,4.237584e+06,4.246184e+06,4.162764e+06,3.955721e+06,4.127032e+06,4.384726e+06,4.584917e+06,4.895828e+06,5.138537e+06,5.405106e+06
5,Orlando-Kissimmee-Sanford,222456.521739,215817.391304,204995.652174,199504.347826,192886.956522,191265.217391,189291.304348,183317.391304,178069.565217,...,1.784000e+06,1.523790e+06,1.495856e+06,1.776956e+06,2.035216e+06,2.203224e+06,2.371634e+06,2.569544e+06,2.783628e+06,2.978733e+06
6,Raleigh-Durham-Chapel Hill,191500.000000,189225.000000,189325.000000,187516.666667,189650.000000,188741.666667,192383.333333,192425.000000,189991.666667,...,2.233067e+06,2.358754e+06,2.298389e+06,2.309690e+06,2.339252e+06,2.362967e+06,2.601515e+06,2.860708e+06,3.095462e+06,3.297587e+06
7,San Francisco-Oakland-Hayward,609295.744681,588831.914894,560665.957447,548961.702128,549729.166667,550302.083333,543089.583333,534577.083333,525085.416667,...,5.991325e+06,6.225602e+06,5.748641e+06,5.982848e+06,7.179224e+06,8.203879e+06,9.411110e+06,1.020174e+07,1.109660e+07,1.224544e+07
8,Seattle-Tacoma-Bellevue,369272.549020,361194.117647,353547.058824,346505.882353,344833.333333,344662.745098,339523.529412,336143.137255,329049.019608,...,3.902489e+06,3.883203e+06,3.507125e+06,3.554526e+06,3.838927e+06,4.062041e+06,4.398627e+06,4.937909e+06,5.616315e+06,6.275696e+06
9,Washington-Arlington-Alexandria,436698.245614,424473.684211,414382.456140,403977.192982,400580.701754,394132.758621,390791.379310,386372.881356,381777.966102,...,4.343010e+06,4.378837e+06,4.269803e+06,4.286233e+06,4.555299e+06,4.675984e+06,4.779969e+06,4.846169e+06,5.041907e+06,5.220421e+06


In [26]:
# price_agg.loc[price_agg.StateName == 'District of Columbia'].ZipCode

In [27]:
# metro_zipcodes = {}
# for metro in metro_names:
#     metro_zipcodes[metro] = list(price_agg.loc[price_agg.Metro == metro].ZipCode)

# metro_zipcodes

In [28]:
# metro_names = ['Seattle-Tacoma-Bellevue',
#  'Washington-Arlington-Alexandria',
#  'Detroit-Warren-Dearborn',
#  'Denver-Aurora-Lakewood',
#  'Austin-Round Rock',
#  'Orlando-Kissimmee-Sanford',
#  'Raleigh-Durham-Chapel Hill',
#  'Dallas-Fort Worth-Arlington',
#  'San Francisco-Oakland-Hayward',
#  'New York-Newark-Jersey City']

# metro_zipcodes = {}
# for metro in metro_names:
#     metro_zipcodes[metro] = list(price_agg.loc[price_agg.Metro == metro].ZipCode)

# zip_geo = {'WA':'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/wa_washington_zip_codes_geo.min.json',
#            'DC':'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/dc_district_of_columbia_zip_codes_geo.min.json', 
#            'MD':'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/md_maryland_zip_codes_geo.min.json', 
#            'VA':'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/va_virginia_zip_codes_geo.min.json', 
#            'WV':'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/wv_west_virginia_zip_codes_geo.min.json', 
#            'MI':'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/mi_michigan_zip_codes_geo.min.json', 
#            'CO':'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/co_colorado_zip_codes_geo.min.json', 
#            'TX':'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/tx_texas_zip_codes_geo.min.json', 
#            'FL':'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/fl_florida_zip_codes_geo.min.json', 
#            'NC':'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/nc_north_carolina_zip_codes_geo.min.json', 
#            'CA':'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ca_california_zip_codes_geo.min.json', 
#            'NY':'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ny_new_york_zip_codes_geo.min.json', 
#            'NJ':'https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/nj_new_jersey_zip_codes_geo.min.json'}
# metro_zips = []

# for key, url in zip_geo.items():
#     r = requests.get(url)
#     zips_json = r.json()
    
#     sub_zips = {}
#     for i in range(0, len(zips_json['features'])):
#         for key, values in metro_zipcodes.items():
#             for value in values:
#                 if value == int(zips_json['features'][i]['properties']['ZCTA5CE10']):
#                     sub_zips[value] = zips_json['features'][i]['geometry']['coordinates'][0]
    
#     metro_zips.append(sub_zips)

In [29]:
# for key, value in metro_zips[1].items():
#     print(key)

In [30]:
# len(metro_zips)

In [31]:
# metro_names

# Create Database Connection

In [32]:
# engine = create_engine('sqlite:///real_estate.sqlite')

In [33]:
# engine.table_names()

In [34]:
# metro_group.to_sql(
#     name = 'median_price_sqft', con = engine,
#     if_exists = 'replace')

In [35]:
# sales_group_median.to_sql(
#     name = 'median_sales_count', con = engine,
#     if_exists = 'replace')

In [36]:
# sales_group_mean.to_sql(
#     name = 'mean_sales_count', con = engine,
#     if_exists = 'replace')

In [37]:
# price_agg.to_sql(
#     name = 'median_price_zip', con = engine,
#     if_exists = 'replace')

In [38]:
# pd.read_sql('SELECT * FROM median_price_zip', con = engine)

# MySQL Connection

In [39]:
import pymysql
pymysql.install_as_MySQLdb()
from mysql_conn import password

ModuleNotFoundError: No module named 'mysql_conn'

In [40]:
# Define database within MySQL client

In [42]:
connection_string = (f"root:martina13@localhost")

engine = create_engine(f"mysql://{connection_string}")
engine.execute("DROP DATABASE IF EXISTS real_estate")
engine.execute("CREATE DATABASE real_estate")

C:\Users\michael\Anaconda3\envs\PythonData\lib\site-packages\pymysql\cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


In [43]:
engine.execute("USE real_estate")
# engine.table_names()

In [44]:
engine.execute("USE real_estate")
price_agg.to_sql(
    name = 'median_price_zip', con = engine,
    if_exists = 'replace', chunksize = 75)
with engine.connect() as con:
    con.execute('ALTER TABLE `median_price_zip` ADD PRIMARY KEY (`index`);')

In [45]:
engine.execute("USE real_estate") # select new db
sales_group_mean.to_sql(
    name = 'mean_sales_count', con = engine,
    if_exists = 'replace')
with engine.connect() as con:
    con.execute('ALTER TABLE `mean_sales_count` ADD PRIMARY KEY (`index`);')

In [46]:
engine.execute("USE real_estate") # select new db
sales_group_median.to_sql(
    name = 'median_sales_count', con = engine,
    if_exists = 'replace')
with engine.connect() as con:
    con.execute('ALTER TABLE `median_sales_count` ADD PRIMARY KEY (`index`);')

In [47]:
engine.execute("USE real_estate") # select new db
metro_group.to_sql(
    name = 'median_price_sqft', con = engine,
    if_exists = 'replace')
with engine.connect() as con:
    con.execute('ALTER TABLE `median_price_sqft` ADD PRIMARY KEY (`index`);')

In [48]:
engine.execute("USE real_estate") # select new db
metro_zips.to_sql(
    name = 'zip_code_coordinates', con = engine,
    if_exists = 'replace')
# with engine.connect() as con:
#     con.execute('ALTER TABLE `median_price_sqft` ADD PRIMARY KEY (`index`);')